In [1]:
from kiara.utils.jupyter import graph_to_image
from kiara.workflow.wrapped import WrappedWorkflow
%load_ext autoreload
%autoreload 2

from kiara import Kiara

kiara = Kiara.instance()

wf = WrappedWorkflow(kiara=kiara, id="my_little_workflow")

onboarding = wf.add_step(module_type="onboarding.folder.import")

onboarding.input.path = "/home/markus/projects/dharpa/notebooks/TopicModelling/data_tm_workflow"

create_table = wf.add_step(module_type="table.from_file_bundle")
create_table.connect_input("files", create_table.output.file_bundle)

extract_date = wf.add_step(module_type="array.map", module_config={"module_type": "date.extract_from_string"}, step_id="extract_date_from_file_name")

extract_date.connect_input("array", create_table, "table.file_name")


NameError: name 'Kiara' is not defined

In [ ]:
graph_to_image(wf.structure.execution_graph)



